In [29]:
import os
import shutil
import pandas as pd
import glob

In [30]:
os.chdir("/mnt")
ddir = f"/mnt/c/Users/youzh/Documents/My documents(not synced)/sP_Workflow"
data_sac = f"{ddir}/Download/SAC"
wdir = f"{ddir}/Download/Verified"
jdir = f"{ddir}/judge"
wsl_dir = f"/home/youzhi/python-projects/sp-project"
os.getcwd()

'/mnt'

# Read saved good signals

In [31]:
ver_dir = wdir
os.chdir(wdir)
df = pd.read_csv(f"{ver_dir}/verified.txt", names=["evt_id","sta", "Tp", 
                                                   "p_len","TsPmP", "sPmP_len",
                                                   "TsPn", "sPn_len", "quality", "evt_num"], header = None)

In [32]:
df

,evt_id,sta,Tp,p_len,TsPmP,sPmP_len,TsPn,sPn_len,quality,evt_num
0,39909423,CI.HOL.,15.558,0.3,18.173,0.3,nan,0.4,a,20220112002458770
1,39909511,AZ.LVA2.,3.723,0.2,11.662,0.3,nan,0.4,b,20220112021208390
2,39909511,AZ.SMER.,12.748,0.2,16.068,0.3,nan,0.4,b,20220112021208390
3,39909511,BC.RMX.,16.035,0.3,19.083,0.3,18.483,0.3,a,20220112021208390
4,39909663,CI.EML.,18.840,0.4,23.763,0.4,22.793,0.4,a,20220112090603170
...,...,...,...,...,...,...,...,...,...,...
100,40183624,CI.SLH.,3.360,0.4,13.400,0.4,nan,0.3,b,20220211232548160
101,40184400,CI.GMA.,5.090,0.4,11.200,0.4,nan,0.3,b,20220212232629910
102,40184400,CI.LUC2.,7.300,0.4,12.100,0.4,nan,0.3,a,20220212232629910
103,40184400,AZ.SND.,12.400,0.4,15.100,0.4,nan,0.3,b,20220212232629910


# Move the good SAC files into a new folder

In [25]:
source_dir = data_sac
target_dir = f"{wsl_dir}/Data/GOOD_SAC_for_readin"

In [18]:
for index, row in df.iterrows():
    prefix = row["sta"].strip()
    suffix = "Z.SAC.bp.cut"
    file_names = os.listdir(os.path.join(source_dir, str(row["evt_num"])))

    # Filter the file names to select only the files with the desired prefix and suffix
    filtered_file_name = [file_name for file_name in file_names if file_name.startswith(prefix) and file_name.endswith(suffix)]


    filepath = os.path.join(source_dir, str(row["evt_num"]), filtered_file_name[0])
    new_filepath = os.path.join(target_dir, str(row["evt_num"]))
    if not os.path.exists(new_filepath):
        os.makedirs(new_filepath)
    shutil.copy(filepath, new_filepath)

# Verify the number of files copied

In [26]:
num_files = 0

# Recursively iterate over all files in the subdirectory and its subdirectories
for root, dirs, files in os.walk(target_dir):
    # For each file, increment the counter
    for file_name in files:
        if file_name.endswith("cut"):
            num_files += 1

print(num_files)

105


# Gather bad signals

In [62]:
folder_path = data_sac

# get a list of all folders in the parent folder
folders = os.listdir(folder_path)

# filter the folders that match the regex pattern and fall within the specified range
matched_folders = []
for folder in folders:
    folder_int = int(folder)
    if 20220112002458770 < folder_int < 20220211232548160:
            matched_folders.append(os.path.join(folder_path, folder))

bad_files = []

for folder in matched_folders:
    evt_num = int(folder.split("/")[-1])
    try:
        handpickP = pd.read_csv(f'{data_sac}/{evt_num}/judge/handpickP.info',delim_whitespace=True,names=["evt","sta", "Tp","TsPmP","TsPn"], header = None)
        
    except: continue
    
    saved_stations = df[df["evt_num"] == evt_num]["sta"].values
    saved_stations = [x.strip() for x in saved_stations]
    stations = glob.glob(f"{folder}/*Z.SAC.bp.cut")
    
    for station in stations:
        station_name = " "+".".join(station.split("/")[-1].split(".")[0:2])+"."
        
        #remove white space for station_name
        station_name = station_name.strip()
        
        if station_name not in saved_stations:
            # checking that the station is not recorded in good signals
            if station_name in handpickP["sta"].values:
                # checkcing if the signal has a handpicked signal
            
                bad_files.append(station)

In [66]:
bad_files[0]

'/mnt/c/Users/youzh/Documents/My documents(not synced)/sP_Workflow/Download/SAC/20220118084437210/CI.CWP..BHZ.SAC.bp.cut'

# move bad signals into a new folder

In [67]:
dst_folder = f"{wsl_dir}/Data/BAD_SAC_for_readin"

for file_path in bad_files:
    file_name = os.path.basename(file_path)
    evt_num = os.path.basename((os.path.split(file_path)[0]))
    dst_path = os.path.join(dst_folder, evt_num)
    if not os.path.exists(dst_path):
        os.makedirs(dst_path)
    dst_file_path = os.path.join(dst_path, file_name)
    
    shutil.copy(file_path, dst_file_path)